In [52]:
import pandas as pd
from Bio import SeqIO 
import os,sys
import sgRNA_utils.sgRNA_primer_util as su
import module.sequencing_primer as sr
import module.from_gbFile_to_seq as fq
import module.parser_input_to_df as pf
import module.product_and_decorate_editingSeq as p_d_seq
import module.order as order
import warnings   
warnings.filterwarnings('ignore')  
import configparser
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import json
from sgRNA_utils.sgRNA_primer_config import config


input_path = '/home/yanghe/program/edit_sequence_design/input/config.json'

In [48]:
seq = 'CACAAATTTCCCATCAACAGTTGGGAAGTAGTGAACTTCCTCTTCAACGACGGAAGAATTATCCCCCCCC'
a = su.find_coordinate_by_pattern(pattern='CAGT', seq=seq)

In [51]:
su.off_target_analysis_from_genome(primer_seq='CCCATCAACAGTTGGGAAGTAGTGAACTTCCTCTTCAACGA',seq=seq, type='RIGHT')

TypeError: off_target_analysis_from_genome() got an unexpected keyword argument 'primer_seq'

In [55]:
help(su.off_target_analysis_from_genome)

Help on function off_target_analysis_from_genome in module sgRNA_utils.sgRNA_primer_util:

off_target_analysis_from_genome()



In [31]:
def off_target_analysis_from_genome(primer_seq, seq, type):

    primer_suc = {}
    result = su.find_coordinate_by_pattern(pattern=primer_seq, seq=seq)

    #若模板脱靶，直接结束
    if result[1] > 1:
        return primer_suc
    else:
    #若模板不脱靶，循环找出15bp不脱靶的序列
        if type == 'RIGHT':
            primer_seq = su.revComp(primer_seq)

        #对每一条引物进行脱靶分析，种子序列为15bp
        j = 0
        for m in range(15,len(primer_seq)+1):                     #控制滑动次数

            pattern = primer_seq[j:m]
            result = su.find_coordinate_by_pattern(pattern=pattern, seq=primer_seq)

            if result[1] > 1:
                if j == len(primer_seq) - 15 + 1 :
                    break
                else:
                    j = j + 1
                    continue
            else:
                coordinate = result[0]['0']

                if type == 'LEFT':
                    primer_suc.update(
                        {
                            'PRIMER_LEFT_0_SEQUENCE':pattern ,
                            'PRIMER_LEFT_0': (coordinate[0] , (coordinate[1] - coordinate[0] + 1) )
                        }
                    )
                elif type == 'RIGHT':
                    primer_suc.update(
                        {
                            'PRIMER_RIGHT_0_SEQUENCE':pattern,
                            'PRIMER_RIGHT_0': ( len(seq) -1 - coordinate[0] , (coordinate[1] - coordinate[0] + 1) )
                        }
                    )
                break
        
        return primer_suc

In [43]:
# primer_seq='CCCATCAACAGTTGGGAAGTAGTGAACTTCCTCTTCAACGA',seq=seq, type='RIGHT'
su.off_target_analysis_from_genome('122')

TypeError: off_target_analysis_from_genome() takes 0 positional arguments but 1 was given

In [ ]:
CACAAATTTCCCATCAACAGTTGGGAAGTAGTGAACTTCCTCTTCAACGACGGAAGAATTATCCCCCCCC
         CCCATCAACAGTTGGGAAGTAGTGAACTTCCTCTTCAACGA

In [58]:
a = su.primer_design('UHA_primer',seq,'left_right')

In [53]:
pd.DataFrame.from_dict(a, orient='index', columns=['Value'])

,Value
PRIMER_LEFT_EXPLAIN,"considered 8, high hairpin stability 1, ok 8"
PRIMER_RIGHT_EXPLAIN,"considered 8, long poly-x seq 8, ok 8"
PRIMER_PAIR_EXPLAIN,"considered 1, ok 1"
PRIMER_LEFT_NUM_RETURNED,1
PRIMER_RIGHT_NUM_RETURNED,1
PRIMER_INTERNAL_NUM_RETURNED,0
PRIMER_PAIR_NUM_RETURNED,1
PRIMER_PAIR_0_PENALTY,5.155524
PRIMER_LEFT_0_PENALTY,4.751098
PRIMER_RIGHT_0_PENALTY,0.404426


# 数据预处理

In [11]:
data = {   
        "chopchop_input": "/home/yanghe/tmp/data_preprocessing/output/info_input.csv",   
        "sgRNA_result_path": "/home/yanghe/tmp/chopchop/output/sgRNA.csv",
        "edit_sequence_design_workdir":"/home/yanghe/tmp/edit_sequence_design/output/",
        "ref_genome":"/home/yanghe/program/data_preprocessing/input/GCA_000011325.1_ASM1132v1_genomic.fna",
        "one_plasmid_file_path": "/home/yanghe/program/edit_sequence_design/input/pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_ccdb_plasmid":"/home/yanghe/program/edit_sequence_design/input/no-ccdb-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",
        "no_sgRNA_plasmid":"/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb",

        "uha_dha_config": {
            "max_right_arm_seq_length": 1050,
            "max_left_arm_seq_length": 1050,
            "min_left_arm_seq_length": 1000,   
            "min_right_arm_seq_length": 1000
        },
        
        "plasmid_label":{
            "ccdb_label":"ccdB",
            "promoter_terminator_label":"gRNA",
            "n_20_label":"N20"
        },

        "primer_json":{
            "primer1":"AACTATTTATCCAGTTGGTACAAAC"
        },
        "region_json":{
            "region1":"tgtgtggaattgtgagcggataacaatttcacacaggaaacagaatt"
        },  

        "sgRNA_primer_json":{
            "primer3":"AACTATTTATCCAGTTGGTACAAAC"
        },
        "ccdb_primer_json":{
            "primer3":"AACTGATTCAGTCTGATTTCGCGGT"
        },
    
        "sgRNA_region_json":{
            "region1":"tgtgtggaattgtgagcggataacaatttcacacaggaaacagaatt"
        },
        
        "ccdb_region_json":{
            "region1":"ATTGTGAGCGGATAACAATTTCACACAGGAAACAGAATTAATTAAGCTTAAAGGAGTTGAGAATGGATAAGAAATACTCAATAGGCTTAGATATCGGCACAAATAGCGTCGGATGGGCGGTGATC"
        },
        
        "enzyme":{
            "enzyme_name":"BsaI",
            "gap_sequence":"A",
            "protection_sequence":"CCA"
        },
        
        "UHA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "SEQ_ALTERED_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,  
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "DHA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "UP_SGRNA_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,
            "PRIMER_MAX_TM": 75,
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },

        "PLASMID_Q_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,  
            "PRIMER_MAX_TM": 75,    
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        "GENOME_Q_ARGS":{
            "PRIMER_OPT_TM": 65,
            "PRIMER_MIN_TM": 55,     
            "PRIMER_MAX_TM": 75,    
            "PRIMER_MIN_GC": 20,
            "PRIMER_MAX_GC": 80
        },
        'sgRNA_result':{
            "Cgl0006_1176_G_A_sub":"1",
            "Cgl2342_213_GCA_ins":"2",
            "Cgl1436_1113_CAA_del":"3",
            "Cgl1790_1647_TCC_sub":"1",
            "Cgl1386_327_18to15_sub":"1",
            "Cgl0591_-1_Ppgk_promoter_ins":"1",
            "Cgl0141_cds_del":"1",
            "153019_ecoil_ybeL_ins":"1",
            "Cgl0851_ecoli_pgi_sub":"1"
        }      
    }

True

In [4]:
    chopchop_input = data['chopchop_input']

    #uha_dha参数
    uha_dha_params = data['uha_dha_config']
    
    # enzyme_path = parent_base_path +'/'+ data['enzyme_path']
    
    sgRNA_result_path = data['sgRNA_result_path']
    one_plasmid_file_path = data['one_plasmid_file_path']
    no_ccdb_plasmid = data['no_ccdb_plasmid']
    no_sgRNA_plasmid = data['no_sgRNA_plasmid']
    
    #plasmid label
    ccdb_label = data['plasmid_label']['ccdb_label']
    promoter_terminator_label = data['plasmid_label']['promoter_terminator_label']
    n_20_label = data['plasmid_label']['n_20_label']   
 
    #primer
    sgRNA_primer_json = data['sgRNA_primer_json']
    ccdb_primer_json = data['ccdb_primer_json']
    
    sgRNA_region_seq_json = data['sgRNA_region_json']
    ccdb_region_seq_json =  data['ccdb_region_json']
    
    #基因组
    genome_path = data['ref_genome']

    #配置引物参数
    # config.S_GLOBAL_ARGS = data['S_GLOBAL_ARGS']
    # config.Q_ARGS = data['Q_ARGS']
    config.UHA_ARGS = data['UHA_ARGS']
    config.SEQ_ALTERED_ARGS = data['SEQ_ALTERED_ARGS']
    config.DHA_ARGS = data['DHA_ARGS']
    config.UP_SGRNA_ARGS = data['UP_SGRNA_ARGS']
    config.DOWN_SGRNA_ARGS = data['DOWN_SGRNA_ARGS']

    config.PLASMID_Q_ARGS = data['PLASMID_Q_ARGS']
    config.GENOME_Q_ARGS = data['GENOME_Q_ARGS']   
    
    #配置输出参数
    output = data['edit_sequence_design_workdir']
    if not os.path.exists(output):
        os.makedirs(output)
     
    enzyme_path = './input/enzyme.csv'  
   

In [5]:
#1.read 编辑序列信息,给chopchop输入加uha、dha信息
info_input_df = read_chopchopInput_add_uha_dha(genome_path, chopchop_input, uha_dha_params)

#2.读取用户填的酶
enzyme_df = su.del_Unnamed(pd.read_csv(enzyme_path))
enzyme = data['enzyme']
enzyme_name = enzyme['enzyme_name']

#3.提取用户选择的sgRNA
selected_sgRNA_result = data['sgRNA_result']
sgRNA = p_d_seq.extract_sgRNA_from_chopchop(sgRNA_result_path, selected_sgRNA_result)

#4.设计源生同源臂引物
uha_dha_primer_df = extract_uha_dha_primer(info_input_df, sgRNA)

#5.提取同源臂
uha_dha_info_primer_df, uha_dha_df, uha_dha_sgRNA_df, info_df = extract_uha_dha(info_input_df, uha_dha_primer_df, sgRNA)


Cgl0006_1176_G_A_sub 1
                   Name                Region  Rank       Target sequence  \
0  Cgl0006_1176_G_A_sub  BA000036.3:6529-6530     1  AACGCGCACGGAAATAACCG   
1  Cgl0006_1176_G_A_sub  BA000036.3:6529-6530     2  CTGTCGTGAAGGCCTGTCCG   
2  Cgl0006_1176_G_A_sub  BA000036.3:6529-6530     3  GCGGTTATTTCCGTGCGCGT   
3  Cgl0006_1176_G_A_sub  BA000036.3:6529-6530     4  GCACGGAAATAACCGCGGAC   

  Genomic location Strand  GC content (%)  Self-complementarity  MM0  MM1  \
0  BA000036.3:6525      -              55                   NaN    0    0   
1  BA000036.3:6508      +              65                   NaN    0    0   
2  BA000036.3:6527      +              60                   NaN    0    0   
3  BA000036.3:6520      -              60                   NaN    0    0   

   MM2  MM3  Efficiency  
0    0    0         NaN  
1    0    0         NaN  
2    0    0         NaN  
3    0    0         NaN  
                   Name                Region  Rank       Target sequence  

{}
{'PRIMER_LEFT_EXPLAIN': 'considered 8, ok 8', 'PRIMER_RIGHT_EXPLAIN': 'considered 408, low tm 132, not in any ok right region 172, ok 104', 'PRIMER_PAIR_EXPLAIN': 'considered 1, ok 1', 'PRIMER_LEFT_NUM_RETURNED': 1, 'PRIMER_RIGHT_NUM_RETURNED': 1, 'PRIMER_INTERNAL_NUM_RETURNED': 0, 'PRIMER_PAIR_NUM_RETURNED': 1, 'PRIMER_PAIR_0_PENALTY': 10.796287109339062, 'PRIMER_LEFT_0_PENALTY': 3.4730231907085454, 'PRIMER_RIGHT_0_PENALTY': 7.323263918630516, 'PRIMER_LEFT_0_SEQUENCE': 'GTTATTTCCGTGCGCGTTGGTGA', 'PRIMER_RIGHT_0_SEQUENCE': 'GCCTGCATTTATAAAGTGCTGACCC', 'PRIMER_LEFT_0': (0, 23), 'PRIMER_RIGHT_0': (1038, 25), 'PRIMER_LEFT_0_TM': 64.52697680929145, 'PRIMER_RIGHT_0_TM': 62.676736081369484, 'PRIMER_LEFT_0_GC_PERCENT': 52.17391304347826, 'PRIMER_RIGHT_0_GC_PERCENT': 48.0, 'PRIMER_LEFT_0_SELF_ANY_TH': 19.610968059260017, 'PRIMER_RIGHT_0_SELF_ANY_TH': 24.27068115227172, 'PRIMER_LEFT_0_SELF_END_TH': 5.11952728907761, 'PRIMER_RIGHT_0_SELF_END_TH': 15.625926837095221, 'PRIMER_LEFT_0_HAIRPIN_TH'

In [13]:
abc = 'AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAGTCGGACTCATCGCAGCCTACGACGGCATCTTCGGACCCGGAACCGGCATGTTCCTCATCATGGCGTTCACCGCACTGCTCTCCCAAAATTTCCTGTCCTCCGCAGCCATGGCGAAGGTCGTAAACACCGCAACAAACCTGGGTGCGCTAATTGTATTCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGCAGTCGCCAATGTCGCAGGCGCACAACTCGGTGCCCGAACGGTGCTTGGTGGCGGTACCAGGCTAATTAGATACGCACTACTAACCCTGGTTGTCGTCATGAGCGTCTACCTCACCTGGCAACAAATCCAAGGAATGTAGATAAGTCGGGGCAAACTCTAAGAAATCATCCTGGTTTGTGGCAGGGAACATTCCTAGGATGGGAGGCATGGCAAAGACAAAACTAGAAAGCATCTCCGAGCGCAACTCGGATGATCCGCTTAGTTACCCCGACCTGACTCTTGCACCAATATCCACAACAGTCACAGTCGTGGTCGCCGAAGACACCCCAAACGATAACTTCTCAGAACTTCTCCTCGGCGCAGCAGAATCAACCCGTCTTTTAGGACTCGACGAGCTCCACATCATCGCTCCATCAGTCCACCTACCCGCTCTCGCAGTAGCAGCCGCCGACATCGCACACCATCTGCCAGAGAAATTCCAATTCTGTGAAGCCGAAACCTGCACTCATCTCCACCCCGACGATGACACCTATCTCACGGCTGAATCCGTTGCTCAATTGGGTACGAAGCTGAAGTCTGCGTAGTTGTTAGGAGCCACCACAGACCCGTAGATTCTTGGGCGGTGTGGTGAATCCTTGCCCTGGAACCTTGCGGGAGACTCTCAGAAATGAGTCATTGGGGCTTGGGGTGTGGTGAAACGCAAACATCCGTCCCCGGACCGCGGTAAAAGGCCGATTTTGATCGAAAATTCCTGCTAATCCGACATTGAAATACAAATACTCGTTCATGAACAAGGTTGCTCAATATTACCGTGAACTGGTTGCGTCACTGAGCGAACGCCTGCGCAATGGCGAACGTGATATCGACGCACTGGTGGAACAGGCGCGCGAGCGCGTAATAAAAACAGGGGAGTTAACGCGAACCGAGGTCGATGAGCTGACGCGAGCTGTCAGACGTGACCTGGAAGAGTTCGCCATGAGCTATGAAGAGAGCCTGAAAGAAGAATCTGACAGCGTCTTTATGCGGGTGATTAAAGAAAGCTTGTGGCAGGAGCTGGCAGACATCACCGATAAAACGCAGCTTGAATGGCGCGAAGTTTTCCAGGACCTCAATCATCATGGGGTTTATCACAGCGGAGAAGTGGTCGGGCTGGGAAATCTGGTCTGCGAGAAATGTCACTTCCATCTCCCGATCTACACACCGGAAGTGCTGACGCTATGCCCGAAATGTGGTCATGACCAGTTCCAGAGACGCCCGTTTGAGCCGTAAGCTTTTAAGTTTTCTCGGTGATCCGTAACGAGAAGTTGAACAGCAAACCCGCAGCAACTGTTCAGAAATCACAACAGTCCCAATGACCTCAAAATCGGCCCCTTCTAGAATCGCCTCTCAGGGCCTTTGAGCGCTAAATTGGGGCAACAGGTCACTGCGTACAAAGAGGGCCTTAAATCGGCTCACATCAAGTCGTGCCAAAACTACATGCGATTCTTGGTGCGCGCAGTGGCAGGAGCGGTCCATGGATCCTCCGGCCACGGGTGTTTGGGGTACCGGCCCCGCATTTCTGCACGGACTTGGCTGTACGGGCCAGACCAGAAACTACGCAGGTCATCGGTAACTGCAAGAGGTCTTCCCGCAGGTGACAGGAGATGAAATTGCACCGGGATTCCGCAGAGCTGCGGGGATTCTTCTAATCCGAAGCATTCTTGTAATTTCACTCGTATAACTGGCCTGCCTGATGAATAATCTAGTCGGTGCTGATTCCCGCTGGGTACGGATAGGTGGGAAGGAGCGAATTCTTCAAAGTTGGTTGCTTCAGGCCAGGGGAGAAGTCGCTGGAGTGCGGGATACATATCGATATTGTTTAGTCGTGTTCCGTGGCTGAGCGCCTCAATTTCTGGAGATAACCACAGGTGCGGGTCCGCAGTTTCAATATCTGGCCAGGGTTCGCCACGGTGTTCGTGGATGAATTTCAGTCGGTCACGCAAGGATGCCGCTTTGTCGGAGAAGTGGAATAGGTCGATTCCGCCTTTGGCAAGGGCTGTGGCGATTGTTTCCGAGGCTTCGGCGGGTGTTGGTTTTGTCGGAGTGGAACTTAATTCGATCGCGCCTGCTGCTTTGACCTTCCGGGCTTGGACTTTTCCGTTGACAAAAATCGCGCGGGTTTCTTCCACGACTCCGATGATGTCTATGGCATCGTCCTCTGAAATGCGGGCGGCGGCGCGGATGATGGCAGAATTTTGTGAGCGGTTAATTGCAGCAACGGAGATCCATTCGGCATCTTTGAGATCTGAATCCATGAGGCGTGCGCGTGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCAATGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGGACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGAAGATCCTTTGATCTTTTCTACGGGGTCTGACGCTCAGTGGAACGAAAACTCACGTTAAGGGATTTTGGTCATGAGATTATCAAAAAGGATCTTCACCTAGATCCTTTTGGGGTGGGCGAAGAACTCCAGCATGAGATCCCCGCGCTGGAGGATCATCCAGCCATTCGGGGTCGTTCACTGGTTCCCCTTTCTGATTTCTGGCATAGAAGAACCCCCGTGAACTGTGTGGTTCCGGGGGTTGCTGATTTTTGCGAGACTTCTCGCGCAATTCCCTAGCTTAGGTGAAAACACCATGAAACACTAGGGAAACACCCATGAAACACCCATTAGGGCAGTAGGGCGGCTTCTTCGTCTAGGGCTTGCATTTGGGCGGTGATCTGGTCTTTAGCGTGTGAAAGTGTGTCGTAGGTGGCGTGCTCAATGCACTCGAACGTCACGTCATTTACCGGGTCACGGTGGGCAAAGAGAACTAGTGGGTTAGACATTGTTTTCCTCGTTGTCGGTGGTGGTGAGCTTTTCTAGCCGCTCGGTAAACGCGGCGATCATGAACTCTTGGAGGTTTTCACCGTTCTGCATGCCTGCGCGCTTCATGTCCTCACGTAGTGCCAAAGGAACGCGTGCGGTGACCACGACGGGCTTAGCCTTTGCCTGCGCTTCTAGTGCTTCGATGGTGGCTTGTGCCTGCGCTTGCTGCGCCTGTAGTGCCTGTTGAGCTTCTTGTAGTTGCTGTTCTAGCTGTGCCTTGGTTGCCATGCTTTAAGACTCTAGTAGCTTTCCTGCGATATGTCATGCGCATGCGTAGCAAACATTGTCCTGCAACTCATTCATTATGTGCAGTGCTCCTGTTACTAGTCGTACATACTCATATTTACCTAGTCTGCATGCAGTGCATGCACATGCAGTCATGTCGTGCTAATGTGTAAAACATGTACATGCAGATTGCTGGGGGTGCAGGGGGCGGAGCCACCCTGTCCATGCGGGGTGTGGGGCTTGCCCCGCCGGTACAGACAGTGAGCACCGGGGCACCTAGTCGCGGATACCCCCCCTAGGTATCGGACACGTAACCCTCCCATGTCGATGCAAATCTTTAACATTGAGTACGGGTAAGCTGGCACGCATAGCCAAGCTAGGCGGCCACCAAACACCACTAAAAATTAATAGTCCCTAGACAAGACAAACCCCCGTGCGAGCTACCAACTCATATGCACGGGGGCCACATAACCCGAAGGGGTTTCAATTGACAACCATAGCACTAGCTAAGACAACGGGCACAACACCCGCACAAACTCGCACTGCGCAACCCCGCACAACATCGGGTCTAGGTAACACTGAGTAACACTGAAATAGAAGTGAACACCTCTAAGGAACCGCAGGTCAATGAGGGTTCTAAGGTCACTCGCGCTAGGGCGTGGCGTAGGCAAAACGTCATGTACAAGATCACCAATAGTAAGGCTCTGGCGGGGTGCCATAGGTGGCGCAGGGACGAAGCTGTTGCGGTGTCCTGGTCGTCTAACGGTGCTTCGCAGTTTGAGGGTCTGCAAAACTCTCACTCTCGCTGGGGGTCATCTCTGGCTGAATTGGAAGTCATGGGCGAACGCCGCATTGAGCTGGCTATTGCTACTAAGAATCACTTGGCGGCGGGTGGCGCGCTCATGATGTTTGTGGGCACTGTTCGACACAACCGCTCACAGTCATTTGCGCAGGTTGAAGCGGGTATTAAGACTGCGTACTCTTCGATGGTGAAAACATCTCAGTGGAAGAAAGAACGTGCACGGTACGGGGTGGAGCACACCTATAGTGACTATGAGGTCACAGACTCTTGGGCGAACGGTTGGCACTTGCACCGCAACATGCTGTTGTTCTTGGATCGTCCACTGTCTGACGATGAACTCAAGGCGTTTGAGGATTCCATGTTTTCCCGCTGGTCTGCTGGTGTGGTTAAGGCCGGTATGGACGCGCCACTGCGTGAGCACGGGGTCAAACTTGATCAGGTGTCTACCTGGGGTGGAGACGCTGCGAAAATGGCAACCTACCTCGCTAAGGGCATGTCTCAGGAACTGACTGGCTCCGCTACTAAAACCGCGTCTAAGGGGTCGTACACGCCGTTTCAGATGTTGGATATGTTGGCCGATCAAAGCGACGCCGGCGAGGATATGGACGCTGTTTTGGTGGCTCGGTGGCGTGAGTATGAGGTTGGTTCTAAAAACCTGCGTTCGTCCTGGTCACGTGGGGCTAAGCGTGCTTTGGGCATTGATTACATAGACGCTGATGTACGTCGTGAAATGGAAGAAGAACTGTACAAGCTCGCCGGTCTGGAAGCACCGGAACGGGTCGAATCAACCCGCGTTGCTGTTGCTTTGGTGAAGCCCGATGATTGGAAACTGATTCAGTCTGATTTCGCGGTTAGGCAGTACGTTCTCGATTGCGTGGATAAGGCTAAGGACGTGGCCGCTGCGCAACGTGTCGCTAATGAGGTGCTGGCAAGTCTGGGTGTGGATTCCACCCCGTGCATGATCGTTATGGATGATGTGGACTTGGACGCGGTTCTGCCTACTCATGGGGACGCTACTAAGCGTGATCTGAATGCGGCGGTGTTCGCGGGTAATGAGCAGACTATTCTTCGCACCCACTAAAAGCGGCATAAACCCCGTTCGATATTTTGTGCGATGAATTTATGGTCAATGTCGCGGGGGCAAACTATGATGGGTCTTGTTGTTGGCGTCCCGGAAAACGATTCCGAAGCCCAACCTTTCATAGAAGGCGGCGGTGGAATTTTTCTCCACATAAGCTGGCAATGTTGCGACGCAACAGGTACAGTGTAATTCACGTTCGCTTTTAAGTTTTCTGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGCTTTTTTTCCAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCCTGAGTAGGACAAATCCGCCGGGAGCGGATTTGAACGTTGCGAAGCAACGGCCCGGAGGGTGGCGGGCAGGACGCCCGCCATAAACTGCCAGGCATCAAATTAAGCAGAAGGCCATCCTGACGGATGGCCTTTTTGCGTTTCTACAAACTCTTTTGTTTATTTTTCTAAATACATTCAAATATGTATCCGCTCATGAATTAATTCCGCTAGATGACGTGCGGCTTCGAAAATCTCGTGATGGCAGGTTGGGCGACCAATAACTGCCTTAAAAAAATTACGCCCCGCCCTGCCACTCATCGCAGTACTGTTGTAATTCATTAAGCATTCTGCCGACATGGAAGCCATCACAGACGGCATGATGAACCTGAATCGCCAGCGGCATCAGCACCTTGTCGCCTTGCGTATAATATTTGCCCATGGTGAAAACGGGGGCGAAGAAGTTGTCCATATTGGCCACGTTTAAATCAAAACTGGTGAAACTCACCCAGGGATTGGCTGAGACGAAAAACATATTCTCAATAAACCCTTTAGGGAAATAGGCCAGGTTTTCACCGTAACACGCCACATCTTGCGAATATATGTGTAGAAACTGCCGGAAATCGTCGTGGTATTCACTCCAGAGCGATGAAAACGTTTCAGTTTGCTCATGGAAAACGGTGTAACAAGGGTGAACACTATCCCATATCACCAGCTCACCGTCTTTCATTGCCATACGGAACTCCGGATGAGCATTCATCAGGCGGGCAAGAATGTGAATAAAGGCCGGATAAAACTTGTGCTTATTTTTCTTTACGGTCTTTAAAAAGGCCGTAATATCCAGCTGAACGGTCTGGTTATAGGTACATTGAGCAACTGACTGAAATGCCTCAAAATGTTCTTTACGATGCCATTGGGATATATCAACGGTGGTATATCCAGTGATTTTTTTCTCCATTTTAGCTTCCTTAGCTCCTGAAAATCTCGTCGAAGCTCGGCGGATTTGTCCTACTCAAGCTGATCCGACAAAATCCACACATTATCCCAGGTGTCCGGATCGGTCAAATACGCTGCCAGCTCATAGACCGTATCCAAAGCATCCGGGGCTGATCCCCGGCGCCAGGGTGGTTTTTCTTTTCACCAGTGAGACGGGCAACAGCTGATTGCCCTTCACCGCCTGGCCCTGAGAGAGTTGCAGCAAGCGGTCCACGTGGTTTGCCCCAGCAGGCGAAAATCCTGTTTGATGGTGGTTAACGGCGGGATATAACATGAGCTGTCTTCGGTATCGTCGTATCCCACTACCGAGATATCCGCACCAACGCGCAGCCCGGACTCGGTAATGGCGCGCATTGCGCCCAGCGCCATCTGATCGTTGGCAACCAGCATCGCAGTGGGAACGATGCCCTCATTCAGCATTTGCATGGTTTGTTGAAAACCGGACATGGCACTCCAGTCGCCTTCCCGTTCCGCTATCGGCTGAATTTGATTGCGAGTGAGATATTTATGCCAGCCAGCCAGACGCAGACGCGCCGAGACAGAACTTAATGGGCCCGCTAACAGCGCGATTTGCTGGTGACCCAATGCGACCAGATGCTCCACGCCCAGTCGCGTACCGTCTTCATGGGAGAAAATAATACTGTTGATGGGTGTCTGGTCAGAGACATCAAGAAATAACGCCGGAACATTAGTGCAGGCAGCTTCCACAGCAATGGCATCCTGGTCATCCAGCGGATAGTTAATGATCAGCCCACTGACGCGTTGCGCGAGAAGATTGTGCACCGCCGCTTTACAGGCTTCGACGCCGCTTCGTTCTACCATCGACACCACCACGCTGGCACCCAGTTGATCGGCGCGAGATTTAATCGCCGCGACAATTTGCGACGGCGCGTGCAGGGCCAGACTGGAGGTGGCAACGCCAATCAGCAACGACTGTTTGCCCGCCAGTTGTTGTGCCACGCGGTTGGGAATGTAATTCAGCTCCGCCATCGCCGCTTCCACTTTTTCCCGCGTTTTCGCAGAAACGTGGCTGGCCTGGTTCACCACGCGGGAAACGGTCTGATAAGAGACACCGGCATACTCTGCGACATCGTATAACGTTACTGGTTTCACATTCACCACCCTGAATTGACTCTCTTCCGGGCGCTATCATGCCATACCGCGAAAGGTTTTGCACCATTCGATGGTGTCAACGTAAATGCCGCTTCGCCTTCGCGCGCGAATTGCAAGCTGATCCGGGCTTATCGACTGCACGGTGCACCAATGCTTCTGGCGTCAGGCAGCCATCGGAAGCTGTGGTATGGCTGTGCAGGTCGTAAATCACTGCATAATTCGTGTCGCTCAAGGCGCACTCCCGTTCTGGATAATGTTTTTTGCGCCGACATCATAACGGTTCTGGCAAATATTCTGAAATGAGCTGTTGACAATTAATCATCGGCTCGTATAATGTGTGGAATTGTGAGCGGATAACAATTTCACACAGGAAACAGAATTAATTAAGCTTAAAGGAGTTGAGAATGGATAAGAAATACTCAATAGGCTTAGATATCGGCACAAATAGCGTCGGATGGGCGGTGATCACTGATGAATATAAGGTTCCGTCTAAAAAGTTCAAGGTTCTGGGAAATACAGACCGCCACAGTATCAAAAAAAATCTTATAGGGGCTCTTTTATTTGACAGTGGAGAGACAGCGGAAGCGACTCGTCTCAAACGGACAGCTCGTAGAAGGTATACACGTCGGAAGAATCGTATTTGTTATCTACAGGAGATTTTTTCAAATGAGATGGCGAAAGTAGATGATAGTTTCTTTCATCGACTTGAAGAGTCTTTTTTGGTGGAAGAAGACAAGAAGCATGAACGTCATCCTATTTTTGGAAATATAGTAGATGAAGTTGCTTATCATGAGAAATATCCAACTATCTATCATCTGCGAAAAAAATTGGTAGATTCTACTGATAAAGCGGATTTGCGCTTAATCTATTTGGCCTTAGCGCATATGATTAAGTTTCGTGGTCATTTTTTGATTGAGGGAGATTTAAATCCTGATAATAGTGATGTGGACAAACTATTTATCCAGTTGGTACAAACCTACAATCAATTATTTGAAGAAAACCCTATTAACGCAAGTGGAGTAGATGCTAAAGCGATTCTTTCTGCACGATTGAGTAAATCAAGACGATTAGAAAATCTCATTGCTCAGCTCCCCGGTGAGAAGAAAAATGGCTTATTTGGGAATCTCATTGCTTTGTCATTGGGTTTGACCCCTAATTTTAAATCAAATTTTGATTTGGCAGAAGATGCTAAATTACAGCTTTCAAAAGATACTTACGATGATGATTTAGATAATTTATTGGCGCAAATTGGAGATCAATATGCTGATTTGTTTTTGGCAGCTAAGAATTTATCAGATGCTATTTTACTTTCAGATATCCTAAGAGTAAATACTGAAATAACTAAGGCTCCCCTATCAGCTTCAATGATTAAACGCTACGATGAACATCATCAAGACTTGACTCTTTTAAAAGCTTTAGTTCGACAACAACTTCCAGAAAAGTATAAAGAAATCTTTTTTGATCAATCAAAAAACGGATATGCAGGTTATATTGATGGGGGAGCTAGCCAAGAAGAATTTTATAAATTTATCAAACCAATTTTAGAAAAAATGGATGGTACTGAGGAATTATTGGTGAAACTAAATCGTGAAGATTTGCTGCGCAAGCAACGGACCTTTGACAACGGCTCTATTCCCCATCAAATTCACTTGGGTGAGCTGCATGCTATTTTGAGAAGACAAGAAGACTTTTATCCATTTTTAAAAGACAATCGTGAGAAGATTGAAAAAATCTTGACTTTTCGAATTCCTTATTATGTTGGTCCATTGGCGCGTGGCAATAGTCGTTTTGCATGGATGACTCGGAAGTCTGAAGAAACAATTACCCCATGGAATTTTGAAGAAGTTGTCGATAAAGGTGCTTCAGCTCAATCATTTATTGAACGCATGACAAACTTTGATAAAAATCTTCCAAATGAAAAAGTACTACCAAAACATAGTTTGCTTTATGAGTATTTTACGGTTTATAACGAATTGACAAAGGTCAAATATGTTACTGAAGGAATGCGAAAACCAGCATTTCTTTCAGGTGAACAGAAGAAAGCCATTGTTGATTTACTCTTCAAAACAAATCGAAAAGTAACCGTTAAGCAATTAAAAGAAGATTATTTCAAAAAAATAGAATGTTTTGATAGTGTTGAAATTTCAGGAGTTGAAGATAGATTTAATGCTTCATTAGGTACCTACCATGATTTGCTAAAAATTATTAAAGATAAAGATTTTTTGGATAATGAAGAAAATGAAGATATCTTAGAGGATATTGTTTTAACATTGACCTTATTTGAAGATAGGGAGATGATTGAGGAAAGACTTAAAACATATGCTCACCTCTTTGATGATAAGGTGATGAAACAGCTTAAACGTCGCCGTTATACTGGTTGGGGACGTTTGTCTCGAAAATTGATTAATGGTATTAGGGATAAGCAATCTGGCAAAACAATATTAGATTTTTTGAAATCAGATGGTTTTGCCAATCGCAATTTTATGCAGCTGATCCATGATGATAGTTTGACATTTAAAGAAGACATTCAAAAAGCACAAGTGTCTGGACAAGGCGATAGTTTACATGAACATATTGCAAATTTAGCTGGTAGCCCTGCTATTAAAAAAGGTATTTTACAGACTGTAAAAGTTGTTGATGAATTGGTCAAAGTAATGGGGCGGCATAAGCCAGAAAATATCGTTATTGAAATGGCACGTGAAAATCAGACAACTCAAAAGGGCCAGAAAAATTCGCGAGAGCGTATGAAACGAATCGAAGAAGGTATCAAAGAATTAGGAAGTCAGATTCTTAAAGAGCATCCTGTTGAAAATACTCAATTGCAAAATGAAAAGCTCTATCTCTATTATCTCCAAAATGGAAGAGACATGTATGTGGACCAAGAATTAGATATTAATCGTTTAAGTGATTATGATGTCGATCACATTGTTCCACAAAGTTTCCTTAAAGACGATTCAATAGACAATAAGGTCTTAACGCGTTCTGATAAAAATCGTGGTAAATCGGATAACGTTCCAAGTGAAGAAGTAGTCAAAAAGATGAAAAACTATTGGAGACAACTTCTAAACGCCAAGTTAATCACTCAACGTAAGTTTGATAATTTAACGAAAGCTGAACGTGGAGGTTTGAGTGAACTTGATAAAGCTGGTTTTATCAAACGCCAATTGGTTGAAACTCGCCAAATCACTAAGCATGTGGCACAAATTTTGGATAGTCGCATGAATACTAAATACGATGAAAATGATAAACTTATTCGAGAGGTTAAAGTGATTACCTTAAAATCTAAATTAGTTTCTGACTTCCGAAAAGATTTCCAATTCTATAAAGTACGTGAGATTAACAATTACCATCATGCCCATGATGCGTATCTAAATGCCGTCGTTGGAACTGCTTTGATTAAGAAATATCCAAAACTTGAATCGGAGTTTGTCTATGGTGATTATAAAGTTTATGATGTTCGTAAAATGATTGCTAAGTCTGAGCAAGAAATAGGCAAAGCAACCGCAAAATATTTCTTTTACTCTAATATCATGAACTTCTTCAAAACAGAAATTACACTTGCAAATGGAGAGATTCGCAAACGCCCTCTAATCGAAACTAATGGGGAAACTGGAGAAATTGTCTGGGATAAAGGGCGAGATTTTGCCACAGTGCGCAAAGTATTGTCCATGCCCCAAGTCAATATTGTCAAGAAAACAGAAGTACAGACAGGCGGATTCTCCAAGGAGTCAATTTTACCAAAAAGAAATTCGGACAAGCTTATTGCTCGTAAAAAAGACTGGGATCCAAAAAAATATGGTGGTTTTGATAGTCCAACGGTAGCTTATTCAGTCCTAGTGGTTGCTAAGGTGGAAAAAGGGAAATCGAAGAAGTTAAAATCCGTTAAAGAGTTACTAGGGATCACAATTATGGAAAGAAGTTCCTTTGAAAAAAATCCGATTGACTTTTTAGAAGCTAAAGGATATAAGGAAGTTAAAAAAGACTTAATCATTAAACTACCTAAATATAGTCTTTTTGAGTTAGAAAACGGTCGTAAACGGATGCTGGCTAGTGCCGGAGAATTACAAAAAGGAAATGAGCTGGCTCTGCCAAGCAAATATGTGAATTTTTTATATTTAGCTAGTCATTATGAAAAGTTGAAGGGTAGTCCAGAAGATAACGAACAAAAACAATTGTTTGTGGAGCAGCATAAGCATTATTTAGATGAGATTATTGAGCAAATCAGTGAATTTTCTAAGCGTGTTATTTTAGCAGATGCCAATTTAGATAAAGTTCTTAGTGCATATAACAAACATAGAGACAAACCAATACGTGAACAAGCAGAAAATATTATTCATTTATTTACGTTGACGAATCTTGGAGCTCCCGCTGCTTTTAAATATTTTGATACAACAATTGATCGTAAACGATATACGTCTACAAAAGAAGTTTTAGATGCCACTCTTATCCATCAATCCATCACTGGTCTTTATGAAACACGCATTGATTTGAGTCAGCTAGGAGGTGACTGAGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTCTAGGTTTATACATAGGCGAGTACTCTGTTATGGAGTCAGATCTTAGCGGCTCTTAAGGAACCGAAGCGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGCTGCAGGTCGACTCTAGAGGATCCCCGGGTACCGAGCTCGAATTCAGCTTGGCTGTTTTGGCGGATGAGAGAAGATTTTCAGCCTGATACAGATTAAATCAGAACGCAGAAGCGGTCTGATAAAACAGAATTTGCCTGGCGGCAGTAGCGCGGTGGTCCCACCTGACCCCATGCCGAACTCAGAAGTGAAACGCCGTAGCGCCGATGGTAGTGTGGGGTCTGCCCATGCGAGAGTAGGGAACTGCCAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCCTGAGTAGGACAAATCCGCCGGGAGCGGATTTGAACGTTGCGAAGCAACGGCCCGGAGGGTGGCGGGCAGGACGCCCGCCATAAACTGCCAGGCATCAAATTAAGCAGAAGGCCATCCTGACGGATGGCCTTTTTGCGTTTCTACAAACTCTTTTGTTTATTTTTCTAAATACATTCAAATATGTATCCGCTCATGAGACAATAACCCTGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGGCATTTTGCCTTCCTGTTTTTGCTCACCCAGAAACGCTGGTGAAAGTAAAAGATGCTGAAGATCAGTTGGGTGCACGAGTGGGTTACATCGAACTGGATCTCAACAGCGGTAAGATCCT'

In [17]:
abc.find('TTGCTCACCCAGAAACGCTG')

13800

In [16]:
su.convert_seq_cor(gb_path='',region_seq_json={'123':'GAGCTGATACCGCTCGCCGC'},strand='+',seq=abc)

给定序列，坐标转成给定序列上的坐标


{'123': '-1,-1'}

In [18]:
asd = pd.read_csv('/home/yanghe/program/plasmid_sequencing_primer_df.csv')

In [19]:
ccd = pd.read_csv('/home/yanghe/program/sequencing_primer_template.csv')

In [39]:
len('AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAGTCGGACTCATCGCAGCCTACGACGGCATCTTCGGACCCGGAACCGGCATGTTCCTCATCATGGCGTTCACCGCACTGCTCTCCCAAAATTTCCTGTCCTCCGCAGCCATGGCGAAGGTCGTAAACACCGCAACAAACCTGGGTGCGCTAATTGTATTCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGCAGTCGCCAATGTCGCAGGCGCACAACTCGGTGCCCGAACGGTGCTTGGTGGCGGTACCAGGCTAATTAGATACGCACTACTAACCCTGGTTGTCGTCATGAGCGTCTACCTCACCTGGCAACAAATCCAAGGAATGTAGATAAGTCGGGGCAAACTCTAAGAAATCATCCTGGTTTGTGGCAGGGAACATTCCTAGGATGGGAGGCATGGCAAAGACAAAACTAGAAAGCATCTCCGAGCGCAACTCGGATGATCCGCTTAGTTACCCCGACCTGACTCTTGCACCAATATCCACAACAGTCACAGTCGTGGTCGCCGAAGACACCCCAAACGATAACTTCTCAGAACTTCTCCTCGGCGCAGCAGAATCAACCCGTCTTTTAGGACTCGACGAGCTCCACATCATCGCTCCATCAGTCCACCTACCCGCTCTCGCAGTAGCAGCCGCCGACATCGCACACCATCTGCCAGAGAAATTCCAATTCTGTGAAGCCGAAACCTGCACTCATCTCCACCCCGACGATGACACCTATCTCACGGCTGAATCCGTTGCTCAATTGGGTACGAAGCTGAAGTCTGCGTAGTTGTTAGGAGCCACCACAGACCCGTAGATTCTTGGGCGGTGTGGTGAATCCTTGCCCTGGAACCTTGCGGGAGACTCTCAGAAATGAGTCATTGGGGCTTGGGGTGTGGTGAAACGCAAACATCCGTCCCCGGACCGCGGTAAAAGGCCGATTTTGATCGAAAATTCCTGCTAATCCGACATTGAAATACAAATACTCGTTCATGAACAAGGTTGCTCAATATTACCGTGAACTGGTTGCGTCACTGAGCGAACGCCTGCGCAATGGCGAACGTGATATCGACGCACTGGTGGAACAGGCGCGCGAGCGCGTAATAAAAACAGGGGAGTTAACGCGAACCGAGGTCGATGAGCTGACGCGAGCTGTCAGACGTGACCTGGAAGAGTTCGCCATGAGCTATGAAGAGAGCCTGAAAGAAGAATCTGACAGCGTCTTTATGCGGGTGATTAAAGAAAGCTTGTGGCAGGAGCTGGCAGACATCACCGATAAAACGCAGCTTGAATGGCGCGAAGTTTTCCAGGACCTCAATCATCATGGGGTTTATCACAGCGGAGAAGTGGTCGGGCTGGGAAATCTGGTCTGCGAGAAATGTCACTTCCATCTCCCGATCTACACACCGGAAGTGCTGACGCTATGCCCGAAATGTGGTCATGACCAGTTCCAGAGACGCCCGTTTGAGCCGTAAGCTTTTAAGTTTTCTCGGTGATCCGTAACGAGAAGTTGAACAGCAAACCCGCAGCAACTGTTCAGAAATCACAACAGTCCCAATGACCTCAAAATCGGCCCCTTCTAGAATCGCCTCTCAGGGCCTTTGAGCGCTAAATTGGGGCAACAGGTCACTGCGTACAAAGAGGGCCTTAAATCGGCTCACATCAAGTCGTGCCAAAACTACATGCGATTCTTGGTGCGCGCAGTGGCAGGAGCGGTCCATGGATCCTCCGGCCACGGGTGTTTGGGGTACCGGCCCCGCATTTCTGCACGGACTTGGCTGTACGGGCCAGACCAGAAACTACGCAGGTCATCGGTAACTGCAAGAGGTCTTCCCGCAGGTGACAGGAGATGAAATTGCACCGGGATTCCGCAGAGCTGCGGGGATTCTTCTAATCCGAAGCATTCTTGTAATTTCACTCGTATAACTGGCCTGCCTGATGAATAATCTAGTCGGTGCTGATTCCCGCTGGGTACGGATAGGTGGGAAGGAGCGAATTCTTCAAAGTTGGTTGCTTCAGGCCAGGGGAGAAGTCGCTGGAGTGCGGGATACATATCGATATTGTTTAGTCGTGTTCCGTGGCTGAGCGCCTCAATTTCTGGAGATAACCACAGGTGCGGGTCCGCAGTTTCAATATCTGGCCAGGGTTCGCCACGGTGTTCGTGGATGAATTTCAGTCGGTCACGCAAGGATGCCGCTTTGTCGGAGAAGTGGAATAGGTCGATTCCGCCTTTGGCAAGGGCTGTGGCGATTGTTTCCGAGGCTTCGGCGGGTGTTGGTTTTGTCGGAGTGGAACTTAATTCGATCGCGCCTGCTGCTTTGACCTTCCGGGCTTGGACTTTTCCGTTGACAAAAATCGCGCGGGTTTCTTCCACGACTCCGATGATGTCTATGGCATCGTCCTCTGAAATGCGGGCGGCGGCGCGGATGATGGCAGAATTTTGTGAGCGGTTAATTGCAGCAACGGAGATCCATTCGGCATCTTTGAGATCTGAATCCATGAGGCGTGCGCG')

2554

In [40]:
asd = 'TGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGGCATTTTGCCTTCCTGTTTTTGCTCACCCAGAAACGCTGGTGAAAGTAAAAGATGCTGAAGATCAGTTGGGTGCACGAGTGGGTTACATCGAACTGGATCTCAACAGCGGTAAGATCCTAAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAGTCGGACTCATCGCAGCCTACGACGGCATCTTCGGACCCGGAACCGGCATGTTCCTCATCATGGCGTTCACCGCACTGCTCTCCCAAAATTTCCTGTCCTCCGCAGCCATGGCGAAGGTCGTAAACACCGCAACAAACCTGGGTGCGCTAATTGTATTCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGCAGTCGCCAATGTCGCAGGCGCACAACTCGGTGCCCGAACGGTGCTTGGTGGCGGTACCAGGCTAATTAGATACGCACTACTAACCCTGGTTGTCGTCATGAGCGTCTACCTCACCTGGCAACAAATCCAAGGAATGTAGATAAGTCGGGGCAAACTCTAAGAAATCATCCTGGTTTGTGGCAGGGAACATTCCTAGGATGGGAGGCATGGCAAAGACAAAACTAGAAAGCATCTCCGAGCGCAACTCGGATGATCCGCTTAGTTACCCCGACCTGACTCTTGCACCAATATCCACAACAGTCACAGTCGTGGTCGCCGAAGACACCCCAAACGATAACTTCTCAGAACTTCTCCTCGGCGCAGCAGAATCAACCCGTCTTTTAGGACTCGACGAGCTCCACATCATCGCTCCATCAGTCCACCTACCCGCTCTCGCAGTAGCAGCCGCCGACATCGCACACCATCTGCCAGAGAAATTCCAATTCTGTGAAGCCGAAACCTGCACTCATCTCCACCCCGACGATGACACCTATCTCACGGCTGAATCCGTTGCTCAATTGGGTACGAAGCTGAAGTCTGCGTAGTTGTTAGGAGCCACCACAGACCCGTAGATTCTTGGGCGGTGTGGTGAATCCTTGCCCTGGAACCTTGCGGGAGACTCTCAGAAATGAGTCATTGGGGCTTGGGGTGTGGTGAAACGCAAACATCCGTCCCCGGACCGCGGTAAAAGGCCGATTTTGATCGAAAATTCCTGCTAATCCGACATTGAAATACAAATACTCGTTCATGAACAAGGTTGCTCAATATTACCGTGAACTGGTTGCGTCACTGAGCGAACGCCTGCGCAATGGCGAACGTGATATCGACGCACTGGTGGAACAGGCGCGCGAGCGCGTAATAAAAACAGGGGAGTTAACGCGAACCGAGGTCGATGAGCTGACGCGAGCTGTCAGACGTGACCTGGAAGAGTTCGCCATGAGCTATGAAGAGAGCCTGAAAGAAGAATCTGACAGCGTCTTTATGCGGGTGATTAAAGAAAGCTTGTGGCAGGAGCTGGCAGACATCACCGATAAAACGCAGCTTGAATGGCGCGAAGTTTTCCAGGACCTCAATCATCATGGGGTTTATCACAGCGGAGAAGTGGTCGGGCTGGGAAATCTGGTCTGCGAGAAATGTCACTTCCATCTCCCGATCTACACACCGGAAGTGCTGACGCTATGCCCGAAATGTGGTCATGACCAGTTCCAGAGACGCCCGTTTGAGCCGTAAGCTTTTAAGTTTTCTCGGTGATCCGTAACGAGAAGTTGAACAGCAAACCCGCAGCAACTGTTCAGAAATCACAACAGTCCCAATGACCTCAAAATCGGCCCCTTCTAGAATCGCCTCTCAGGGCCTTTGAGCGCTAAATTGGGGCAACAGGTCACTGCGTACAAAGAGGGCCTTAAATCGGCTCACATCAAGTCGTGCCAAAACTACATGCGATTCTTGGTGCGCGCAGTGGCAGGAGCGGTCCATGGATCCTCCGGCCACGGGTGTTTGGGGTACCGGCCCCGCATTTCTGCACGGACTTGGCTGTACGGGCCAGACCAGAAACTACGCAGGTCATCGGTAACTGCAAGAGGTCTTCCCGCAGGTGACAGGAGATGAAATTGCACCGGGATTCCGCAGAGCTGCGGGGATTCTTCTAATCCGAAGCATTCTTGTAATTTCACTCGTATAACTGGCCTGCCTGATGAATAATCTAGTCGGTGCTGATTCCCGCTGGGTACGGATAGGTGGGAAGGAGCGAATTCTTCAAAGTTGGTTGCTTCAGGCCAGGGGAGAAGTCGCTGGAGTGCGGGATACATATCGATATTGTTTAGTCGTGTTCCGTGGCTGAGCGCCTCAATTTCTGGAGATAACCACAGGTGCGGGTCCGCAGTTTCAATATCTGGCCAGGGTTCGCCACGGTGTTCGTGGATGAATTTCAGTCGGTCACGCAAGGATGCCGCTTTGTCGGAGAAGTGGAATAGGTCGATTCCGCCTTTGGCAAGGGCTGTGGCGATTGTTTCCGAGGCTTCGGCGGGTGTTGGTTTTGTCGGAGTGGAACTTAATTCGATCGCGCCTGCTGCTTTGACCTTCCGGGCTTGGACTTTTCCGTTGACAAAAATCGCGCGGGTTTCTTCCACGACTCCGATGATGTCTATGGCATCGTCCTCTGAAATGCGGGCGGCGGCGCGGATGATGGCAGAATTTTGTGAGCGGTTAATTGCAGCAACGGAGATCCATTCGGCATCTTTGAGATCTGAATCCATGAGGCGTGCGCGTGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTA'

In [37]:
asd.find('GAGCTGATACCGCTCGCCGC')

-1

In [32]:
op = ccd.loc[0,'plasmid'].upper()

In [33]:
op.find('GAGCTGATACCGCTCGCCGC')

-1

In [44]:
ast = 'GAGCTGATACCGCTCGCCGC'

In [50]:
a_dict = {'a':123,'b':'345','c':"789"}
b_dict = {'a':456,'b':'789'}

In [51]:
a_dict.update(b_dict)

In [52]:
a_dict

{'a': 456, 'b': '789', 'c': '789'}

# 双质粒设计

## 构建两个新质粒

In [10]:
no_ccdb_uha_dha_sgRNA_df,\
sgRNA_plasmid_backbone,\
promoter_seq,\
terminator_seq,\
sgRNA_promoter_terminator= p_d_seq.create_new_plasmid(no_ccdb_plasmid,uha_dha_sgRNA_df.copy(),ccdb_label, promoter_terminator_label, n_20_label)


no_sgRNA_uha_dha_ccdb_df,\
ccdB_plasmid_backbone,\
ccdB_promoter_terminator_up_seq  = p_d_seq.create_new_plasmid(no_sgRNA_plasmid,uha_dha_sgRNA_df.copy(),ccdb_label, promoter_terminator_label, n_20_label)

-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


In [11]:
#酶切退火方式获得质粒和n20
enzymeCutSeq_and_N20_df = two_plasmid_system_n20_enzyme_cut_seq(no_ccdb_uha_dha_sgRNA_df,promoter_seq,enzyme_df,enzyme_name)

In [9]:
#选择引物设计类型
#质粒引物的设计类型：1---用户指定范围，2----无需用户指定范围，3----用户指定额外引物

In [12]:
plasmid_primer_desgin_type = 1

## 给定质粒的区域设计引物

In [13]:
sgRNA_region_json = {'region1': '371,570', 'region2': '11307,200'}

In [14]:
#将sgRNA质粒分割成：promoter_terminator_up_seq、promoter_seq、n20_coordinate_seq、terminator_seq、promoter_terminator_down_seq
sgRNA_plasmid_seq, sgRNA_plasmid_region_seq = p_d_seq.plasmid_region_division_by_labels(no_ccdb_plasmid,
                                                                                        ccdb_label,
                                                                                        promoter_terminator_label,
                                                                                        n_20_label)
#将ccdb质粒分割成：ccdb_up_seq、ccdb、ccdb_down_seq
ccdb_plasmid_seq, ccdb_plasmid_region_seq = p_d_seq.plasmid_region_division_by_labels(no_sgRNA_plasmid,
                                                                                      ccdb_label,
                                                                                      promoter_terminator_label,
                                                                                      n_20_label)


-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


In [15]:
#针对sgRNA质粒的操作：
#确定第一条引物的位置在质粒上的位置：先找到启动子终止子的绝对位置，再确定相对位置，再确定绝对位置
promoter_terminator_seq = sgRNA_plasmid_region_seq['promoter_seq']+sgRNA_plasmid_region_seq['n20_coordinate_seq']+sgRNA_plasmid_region_seq['terminator_seq']
promoter_terminator_start = sgRNA_plasmid_seq.find(promoter_terminator_seq)
promoter_terminator_end = promoter_terminator_start + len(promoter_terminator_seq)
first_primer_position_in_promoter_terminator = promoter_terminator_seq.find(sgRNA_plasmid_region_seq['terminator_seq'])
first_primer_start_position = promoter_terminator_start + first_primer_position_in_promoter_terminator
#将用户提供的质粒设计区域转换成，各个区域之间的最小，最大距离
sgRNA_plasmid_seq_len = len(sgRNA_plasmid_seq)
sgRNA_distance_dict = region_2_distance(sgRNA_plasmid_seq_len, sgRNA_region_json,first_primer_start_position)


{'primer1': (2591, -8516), 'primer2': (11517, 11716)}


In [16]:
#针对区域设计引物，特殊处理第一条和最后一条引物
sgRNA_plasmid_primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(first_primer_start_position, sgRNA_plasmid_seq, sgRNA_distance_dict, 20)
sgRNA_plasmid_primer_df =  su.result_primer_list_to_df(sgRNA_plasmid_primer_result_list)

OSError: Illegal element in PRIMER_PRODUCT_SIZE_RANGE

In [122]:
sgRNA_plasmid_primer_df

,Region,primer_f_seq_(5'-3'),primer_r_seq_(5'-3'),primer_f_Tm,primer_r_Tm,product_size,product_value
0,1,GTTTTAGAGCTAGAAATAGCAAGT,CGCTGTCTCTCCACTGTCAA,54.971016,59.684749,2830,GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTAT...
0,2,GAAGCGACTCGTCTCAAACGGACA,TGTCCACATCACTATTATCAGGA,65.080717,56.526896,370,GAAGCGACTCGTCTCAAACGGACAGCTCGTAGAAGGTATACACGTC...
0,3,AACTATTTATCCAGTTGGTACAAAC,TGAATTACACTGTACCTGTTGCGT,55.566353,60.741235,8126,GAAGCGACTCGTCTCAAACGGACAGCTCGTAGAAGGTATACACGTC...


In [ ]:
Index(['ID', 'PRIMER_LEFT_WHOLE_SEQUENCE', 'PRIMER_RIGHT_WHOLE_SEQUENCE',       'PRODUCT_SEQUENCE', 'PRODUCT_WHOLE_LENGTH'],      dtype='object')

In [101]:
first_primer_start_position

8716

In [102]:
    distance_dict = {}
    for k,v in sgRNA_region_json.items():
        arr = v.split(',')
        first = int(arr[0])
        last = int(arr[1])
        if first  > first_primer_start_position:
            distance = first - first_primer_start_position
        else:
            distance = sgRNA_plasmid_seq_len - first_primer_start_position + first
        distance_dict.update({distance:v})
    sorted_distance = sorted(distance_dict.keys())

In [33]:
    gb = SeqIO.read(no_ccdb_plasmid, "genbank")
    gb_seq = str(gb.seq)
    #get coordinate
    ccdb_coordinate = su.get_feature_coordinate(ccdb_label,no_ccdb_plasmid)
    promoter_terminator_coordinate = su.get_feature_coordinate(promoter_terminator_label,one_plasmid_file_path)  
    #N20
    n20_coordinate = su.get_feature_coordinate(n_20_label,one_plasmid_file_path)


-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8949 9413
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
9002 9022


In [35]:
ccdb_coordinate==(-1, -1)

True

In [104]:
distance_dict

{3001: '371,570', 2591: '11307,200'}

In [ ]:
def two_plasmid_system_design_by_no_user(no_ccdb_uha_dha_sgRNA_df,ccdB_plasmid_backbone,enzyme_df,enzyme_name):
    #设计sgRNA质粒引物
    no_ccdb_primer_template_df = no_ccdb_uha_dha_sgRNA_df[['Name','Region','sgRNA_template','Rev Target sequence']]
    no_ccdb_primer_template_df['Region'] = no_ccdb_primer_template_df['Name'] + ';' + no_ccdb_primer_template_df['Region']
    sgRNA_plasmid_primer_df = p_d_seq.design_primer(no_ccdb_primer_template_df,'Region','sgRNA_template','sgRNA')
    no_ccdb_uha_dha_sgRNA_df['Region'] = no_ccdb_uha_dha_sgRNA_df['Name'] +';'+ no_ccdb_uha_dha_sgRNA_df['Region']
    
    #sgRNA质粒引物加接头
    sgRNA_plasmid_primer_df = pd.merge(no_ccdb_uha_dha_sgRNA_df[['Region','Target sequence','Rev Target sequence']],sgRNA_plasmid_primer_df,on=['Region'],how='inner')
    sgRNA_plasmid_primer_df = p_d_seq.add_joint_sgRNA_primer(sgRNA_plasmid_primer_df,enzyme_df,enzyme_name,stype='sgRNA_plasmid_primer_joint')
    
    #设计ccdB质粒引物
    no_sgRNA_primer_template_df = pd.DataFrame(columns=['plasmid_backbone_primer','plasmid_backbone'],data=[[f'ccdb_plasmid;primer',ccdB_plasmid_backbone]])
    ccdb_plasmid_primer_df = p_d_seq.design_primer(no_sgRNA_primer_template_df,'plasmid_backbone_primer','plasmid_backbone','plasmid')
    
    #ccdb质粒引物加接头
    ccdb_plasmid_primer_df = p_d_seq.add_joint_sgRNA_primer(ccdb_plasmid_primer_df,enzyme_df,enzyme_name,stype='ccdb_plasmid_primer_joint')
    
    #提取引物的必要部分
    sgRNA_plasmid_p_df = sgRNA_plasmid_primer_df[['Region',r"primer_f_seq_(5'-3')_joint",r"primer_r_seq_(5'-3')_joint","product_value_joint","product_size_joint"]]
    ccdb_plasmid_p_df = ccdb_plasmid_primer_df[['Region',r"primer_f_seq_(5'-3')_joint",r"primer_r_seq_(5'-3')_joint","product_value_joint","product_size_joint"]]

    return sgRNA_plasmid_p_df, ccdb_plasmid_p_df, sgRNA_plasmid_primer_df, ccdb_plasmid_primer_df

In [9]:
df = pd.read_csv('/home/yanghe/program/123.csv')

In [40]:
df.loc[1,'PRIMER_RIGHT_WHOLE_SEQUENCE']

'GGTCCAGAGTAGAAAACTTAAAAGCGAACGTGAATTACACTGTACCTGTTGCGT'

In [37]:
'CCAGGTCTCATTCTGTTTTAGAGCTAGAAATAGCAAGT'

,ID,0_PRIMER_LEFT_WHOLE_SEQUENCE,0_PRIMER_RIGHT_WHOLE_SEQUENCE,1_PRIMER_LEFT_WHOLE_SEQUENCE,1_PRIMER_RIGHT_WHOLE_SEQUENCE
0,153019_ecoil_ybeL_ins;BA000036.3:153019-153020,CCAGGTCTCATTCTGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTAGAAAACTTAAAAGCGAACGTGAATTACACTGTACC...
1,Cgl0006_1176_G_A_sub;BA000036.3:6529-6530,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTCGGTTATTTCCGTGCGCGTTTGAATTACACTGTACC...
2,Cgl0141_cds_del;BA000036.3:153222-155613,CCAGGTCTCAACCGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTCGGTGTGTTAATCCGCCAGCTGAATTACACTGTACC...
3,Cgl0591_-1_Ppgk_promoter_ins;BA000036.3:604469...,CCAGGTCTCAGCAGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTCTGCGCGTAACCTGGGAAGCTGAATTACACTGTACC...
4,Cgl0851_ecoli_pgi_sub;BA000036.3:907755-909378,CCAGGTCTCATCCGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTCGGAGAAGGTGTACTTCTCGTGAATTACACTGTACC...
5,Cgl1386_327_18to15_sub;BA000036.3:1454441-1454459,CCAGGTCTCAGCTAGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTTAGCCACCTTCTGATTCTTCTGAATTACACTGTACC...
6,Cgl1436_1113_CAA_del;BA000036.3:1514089-1514092,CCAGGTCTCAGTGGGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTCCACTTTGCAGACGGAGGGTTGAATTACACTGTACC...
7,Cgl1790_1647_TCC_sub;BA000036.3:1899017-1899020,CCAGGTCTCAATTAGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTGTTATCCGCTCACAATTCCACACA,CCAGGTCTCATAACAATTTCACACAGGAAACAGA,GGTCCAGAGTTAATCTTATTGACTTCTTCTTGAATTACACTGTACC...


In [19]:
df1 = pd.read_csv('/home/yanghe/program/1234.csv')

In [16]:
df2 = pd.read_csv('/home/yanghe/program/456.csv')

In [24]:
df1.loc[:0]

,Region,primer_f_seq_(5'-3')_joint,primer_r_seq_(5'-3')_joint,product_value_joint,product_size_joint
0,153019_ecoil_ybeL_ins;BA000036.3:153019-153020,CCAGGTCTCATTCTGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCATTCTGTTTTAGAGCTAGAAATAGCAAGTTAAAATAA...,7711


In [17]:
df2

,Region,primer_f_seq_(5'-3')_joint,primer_r_seq_(5'-3')_joint,product_value_joint,product_size_joint
0,1,CCAGGTCTCAGTTTTAGAGCTAGAAATAGCAAGT,GGTCCAGAGTAGTTTGTCCACATCACTATTATCAGGA,CCAGGTCTCAGTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCT...,3224
1,2,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAAC,GGTCCAGAGTAGGATCTTACCGCTGTTGAGATCCA,CCAGGTCTCAAACTATTTATCCAGTTGGTACAAACCTACAATCAAT...,4507


In [7]:
dfg = pd.merge(sgRNA_tsv_df,ccdb_tsv_df,on='name',suffixes=['_SGRNA', '_CCDB'])

In [8]:
dfg.columns

Index(['name', 'sgRNA_gb', '0_PRIMER_LEFT_WHOLE_SEQUENCE_sgRNA',
       '0_PRIMER_RIGHT_WHOLE_SEQUENCE_sgRNA',
       '1_PRIMER_LEFT_WHOLE_SEQUENCE_sgRNA',
       '1_PRIMER_RIGHT_WHOLE_SEQUENCE_sgRNA', 'ccdb_gb',
       'UHA_PRIMER_LEFT_WHOLE_SEQUENCE', 'UHA_PRIMER_RIGHT_WHOLE_SEQUENCE',
       'SEQ_PRIMER_LEFT_WHOLE_SEQUENCE', 'SEQ_PRIMER_RIGHT_WHOLE_SEQUENCE',
       'DHA_PRIMER_LEFT_WHOLE_SEQUENCE', 'DHA_PRIMER_RIGHT_WHOLE_SEQUENCE',
       '0_PRIMER_LEFT_WHOLE_SEQUENCE_ccdb',
       '0_PRIMER_RIGHT_WHOLE_SEQUENCE_ccdb',
       '1_PRIMER_LEFT_WHOLE_SEQUENCE_ccdb',
       '1_PRIMER_RIGHT_WHOLE_SEQUENCE_ccdb'],
      dtype='object')

In [10]:



#针对ccdb载体质粒的操作：
ccdb_start = ccdb_plasmid_seq.find(ccdb_plasmid_region_seq['ccdb'])
ccdb_end = ccdb_start + len(ccdb_plasmid_region_seq['ccdb'])
first_primer_start_position = ccdb_end
ccdb_plasmid_seq_len = len(ccdb_plasmid_seq)
ccdb_distance_dict = region_2_distance(ccdb_plasmid_seq_len, ccdb_region_json,first_primer_start_position)
ccdb_plasmid_primer_result_list = p_d_seq.design_primer_by_region_in_plasmid(first_primer_start_position,ccdb_plasmid_seq, ccdb_distance_dict,len(ccdb_plasmid_region_seq['ccdb']))
ccdb_plasmid_primer_df = su.result_primer_list_to_df(ccdb_plasmid_primer_result_list)

#sgRNA质粒引物加接头
sgRNA_plasmid_primer_joint_df = p_d_seq.sgRNA_sgRNAprimer_merge(uha_dha_sgRNA_df, sgRNA_plasmid_primer_df)
sgRNA_primers_sum=len(sgRNA_plasmid_primer_df)
sgRNA_plasmid_primer_joint_df = p_d_seq.add_joint_plasmid_primer(enzyme_df,
                                                                 enzyme_name,
                                                                 sgRNA_plasmid_primer_joint_df,
                                                                 sgRNA_primers_sum,
                                                                 primer_type='sgRNA')

-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
8643 9107
-1 -1
-1 -1
-1 -1
-1 -1
8696 8716
tag: 0 marker -1 -1
-1 -1 8643 9107
kjfdhsgjkhd
-1 -1
5057 5363
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
tag: 0 marker -1 -1
-1 -1 5057 5363
kjfdhsgjkhd


## 给定质粒上的引物

In [17]:
#根据给定sgRNA质粒设计
uha_dha_sgRNA_df, sgRNA_plasmid_backbone, sgRNA_promoter_terminator, sgRNA_primer_json,enzyme_df,enzyme_name,n_20_label,ccdb_label,promoter_terminator_label

(                           Name                      Region  \
 0         153019_ecoil_ybeL_ins    BA000036.3:153019-153020   
 1          Cgl0006_1176_G_A_sub        BA000036.3:6529-6530   
 2               Cgl0141_cds_del    BA000036.3:153222-155613   
 3  Cgl0591_-1_Ppgk_promoter_ins    BA000036.3:604469-604470   
 4         Cgl0851_ecoli_pgi_sub    BA000036.3:907755-909378   
 5        Cgl1386_327_18to15_sub  BA000036.3:1454441-1454459   
 6          Cgl1436_1113_CAA_del  BA000036.3:1514089-1514092   
 7          Cgl1790_1647_TCC_sub  BA000036.3:1899017-1899020   
 
                                                  UHA  UHA_size  \
 0  AAAGCCCTGCCCACCGGATGGAAACGCTGGGCCGCCATCGTTGCAG...      1034   
 1  AAGCGCCCGGGCATGTACATCGGTTCAACTGGACCGCGTGGACTGC...      1026   
 2  TCATCATCGGCGGCCACATGTGGTGGACCCTAGGACTCGTGCTGGC...      1035   
 3  GGAATCACAGGCAGGGTGGGAGATCTTCCACGCCTTGCAGGTGAAA...      1050   
 4  AACCCAAACCCTGCCCGAAGGAACTCAACGCAGAAGCACCAGCACT...      1036   
 5  GGCCCAATGTGGTCCC

In [18]:
sgRNA_primer_position_json, sgRNA_failture_primer = p_d_seq.check_locate_primer(sgRNA_plasmid_backbone, sgRNA_primer_json)
ccdb_primer_position_json, ccdb_failture_primer = p_d_seq.check_locate_primer(ccdB_plasmid_backbone, ccdb_primer_json)

In [19]:
sgRNA_promoter_terminator_start = sgRNA_plasmid_backbone.find(sgRNA_promoter_terminator)

In [20]:
sort_compose_primer(sgRNA_promoter_terminator_start,
                    sgRNA_primer_json,
                    sgRNA_primer_position_json,
                    no_ccdb_plasmid,
                    sgRNA_plasmid_backbone,
                    n_20_label,
                    ccdb_label,
                    promoter_terminator_label)

NameError: name 'first_left_last_right_primer_design' is not defined

In [54]:
sgRNA_region_seq_json = {
    'region1':"tgtgtggaattgtgagcggataacaatttcacacaggaaacagaatt",
    'region2':"aaagatgctgaagatcagttgggtgcacgagtgggttacatcgaactggatctcaacagcggtaagatccttgagagttttcgccccgaagaacgttttccaatgatgagcacttttgcttcctcgctcactgact"
}


In [55]:

sgRNA_gb_path = '/home/yanghe/program/edit_sequence_design/input/no-ccdb-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'
ccdb_gb_path = '/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'


def convert(gb_path,region_seq_json):
    gb = SeqIO.read(gb_path, "genbank")
    gb_seq = str(gb.seq).upper()
    
    region_cor_json = {}
    for k,v in region_seq_json.items():
        v = v.upper()
        start = gb_seq.find(v)
        end = start + len(v)
        if len(gb_seq) < end:
            end = end - len(gb_seq)
        region_cor_json.update({k:f'{start},{end}'})
        
    return region_cor_json

In [56]:
sgRNA_region_json = convert(sgRNA_gb_path,sgRNA_region_seq_json)

In [57]:
sgRNA_region_json

{'region1': '11299,11346', 'region2': '4986,5122'}

In [20]:
ccdb_region_json = convert(sgRNA_gb_path,ccdb_region_seq_json)

In [21]:
ccdb_region_json

{'region1': '-1,130', 'region2': '-1,152'}

In [15]:
ccdb_gb_path = '/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'

gb = SeqIO.read(ccdb_gb_path, "genbank")
gb_seq = str(gb.seq)

In [ ]:
TCATCGACTT

In [75]:
su.revComp('ATTTCACAC')[::-1]

'TAAAGTGTG'

In [62]:
region_json = {'region1': 'ATTTCACAC'}

In [73]:
region_json = {'region1': 'GTGTGAAAT'}

In [114]:
region_json = {'region1': 'ATTTCACAC','region2':'dhbsjkhb'}

'CAATTTCACACAGGAAACAGAATTAATTAAGCTTAAAGGAGTTGAGAATGGATAAGAAATA'

In [100]:
cor_json_plus

{'region1': '-1,-1'}

In [98]:
gb_path =  '/home/yanghe/program/edit_sequence_design/input/no-sgRNA-pXMJ19-Cas9A-gRNA-crtYEb-Ts - ori.gb'

cor_json_plus = convert_seq_cor(gb_path,region_json,strand='+')
cor_json_min = convert_seq_cor(gb_path,region_json,strand='-')

In [115]:

aa = check_seq_in_gb(gb_path,region_json)   

In [116]:
UHA_ARGS
SEQ_ALTERED_ARGS
DHA_ARGS
UP_SGRNA_ARGS
DOWN_SGRNA_ARGS

PLASMID_Q_ARGS
GENOME_Q_ARGS

,region1,region2
0,ATTTCACAC,The sequence you provided is not on the plasmi...


In [105]:
def check_seq_in_gb(gb_path,seq_json):
    
    cor_json_plus = convert_seq_cor(gb_path,seq_json,strand='+')
    cor_json_min = convert_seq_cor(gb_path,seq_json,strand='-')
    
    for plus,minus in zip(cor_json_plus,cor_json_min):
        if cor_json_plus[plus] =='-1,-1' and  cor_json_min[minus] == '-1,-1':
            #序列不存在GB文件
             region_json[plus] = 'The sequence you provided is not on the plasmid file'
        elif cor_json_plus[plus]!='-1,-1' and cor_json_min[minus] == '-1,-1':
            #序列在正义链
            pass
        elif cor_json_plus[plus]!='-1,-1' and cor_json_min[minus] == '-1,-1':
            #序列在负义链
            region_json[plus] = su.revComp(region_json[plus])
        else:
            pass
    return seq_json

In [84]:
#只要不存在off-target，可义将一切序列转化成坐标
def convert_seq_cor(gb_path,region_seq_json,strand='+'):
    gb = SeqIO.read(gb_path, "genbank")
    gb_seq = str(gb.seq).upper()
    
    if strand == '-':
        gb_seq = su.revComp(gb_seq)
    
    region_cor_json = {}
    for k,v in region_seq_json.items():
        v = v.upper()
        start = gb_seq.find(v)
        end = start + len(v)
        if start == -1:
            i = 1
            start_seq = ''
            end_seq = ''
            while True:
                new_start = gb_seq.find(v[:i])
                if new_start == -1:
                    break
                else:
                    if gb_seq.find(v[i:]) == 0:
                        start_seq = v[:i]
                        end_seq = v[i:]
                        print(start_seq,end_seq)
                        break
                    else:        
                        i = i + 1
                        if i == len(v):
                            break
            if start_seq == '' and end_seq == '':
                start = -1
                end = -1
            else:
                start = gb_seq.find(start_seq)
                end = gb_seq.find(end_seq)
                end = end + len(v)
                
        region_cor_json.update({k:f'{start},{end}'})
        
    return region_cor_json  